# train YOLO

In [3]:
from ultralytics import YOLO

In [2]:
import torch

device = 'cpu'
# print('mps', torch.backends.mps.is_available())
if torch.cuda.is_available():
    device = '0'
device

'cpu'

In [5]:
model = YOLO("yolov8n-seg.pt")  # load a pretrained model
model.train(data="/Users/samedi/Documents/Coding/swans/dataset/swans.yaml",# signgle_cls=True,
            epochs=50, 
            device=device, 
            imgsz=320,
            resume=False)
success = model.export(format="onnx")  # export the model to ONNX format
metrics = model.val()

New https://pypi.org/project/ultralytics/8.0.111 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.110 🚀 Python-3.10.6 torch-2.0.1 CPU
yolo/engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=swans.yaml, epochs=50, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=to

KeyboardInterrupt: 

In [4]:
model = YOLO('yolov8m-cls.pt')
model.train(data="/Users/samedi/Documents/Coding/swans/dataset/", 
            epochs=50, 
            device=device, 
            imgsz=320,
            resume=False)
success = model.export(format="onnx")  # export the model to ONNX format
metrics = model.val()

Ultralytics YOLOv8.0.111 🚀 Python-3.10.6 torch-2.0.1 CPU
yolo/engine/trainer: task=classify, mode=train, model=yolov8m-cls.pt, data=/Users/samedi/Documents/Coding/swans/dataset/, epochs=50, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False,

KeyboardInterrupt: 

# train CatBoost

In [127]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, metrics
from sklearn.metrics import accuracy_score

In [91]:
number = 5
cols = ['klikun p','maliy p','shipun p','seg conf','cls conf','mul conf','pred']
columns = [ col+' '+str(i+1) for i in range(number) for col in cols ] + ['seg max', 'cls max', 'common']
print(columns)

['klikun p 1', 'maliy p 1', 'shipun p 1', 'seg conf 1', 'cls conf 1', 'mul conf 1', 'pred 1', 'klikun p 2', 'maliy p 2', 'shipun p 2', 'seg conf 2', 'cls conf 2', 'mul conf 2', 'pred 2', 'klikun p 3', 'maliy p 3', 'shipun p 3', 'seg conf 3', 'cls conf 3', 'mul conf 3', 'pred 3', 'klikun p 4', 'maliy p 4', 'shipun p 4', 'seg conf 4', 'cls conf 4', 'mul conf 4', 'pred 4', 'klikun p 5', 'maliy p 5', 'shipun p 5', 'seg conf 5', 'cls conf 5', 'mul conf 5', 'pred 5', 'seg max', 'cls max', 'common']


In [92]:
def preprocess(df: pd.DataFrame):
    df['mul conf']=df['seg conf']*df['cls conf']

    y = df.groupby('path')['true'].first()
    x = []

    for path, r in y.items():
        sdf = df[df.path == path].drop(columns=['path','true'])
        sdf['size'] = sdf['width']*sdf['height']
        sdf = sdf.sort_values(['mul conf','size'],ascending=False).drop(columns=['width','height','size'])

        agr = [
            sdf.loc[sdf['seg conf'].idxmax()]['pred'],
            sdf.loc[sdf['cls conf'].idxmax()]['pred'],
            sdf['pred'].value_counts().idxmax()
        ]

        ar = sdf[cols].values.flatten()[:len(cols)*number].copy()
        ar.resize(len(cols)*number)

        x.append(ar.tolist() + agr)

    return pd.DataFrame(data=x,columns=columns), y

In [94]:
X_train, y_train = preprocess(pd.read_csv('train.csv'))
X_val, y_val = preprocess(pd.read_csv('val.csv'))
X_test, y_test = preprocess(pd.read_csv('test.csv'))
X_submit, y_submit = preprocess(pd.read_csv('sumbit.csv'))

In [97]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(pd.concat([X_train, X_val]), pd.concat([y_train, y_val]), train_size=0.85, random_state=0)

In [98]:
model = CatBoostClassifier(random_seed=0,logging_level='Silent')

In [118]:
cat = [i*len(cols)+6 for i in range(number)]+[len(columns)-3, len(columns)-2, len(columns)-1]

In [108]:
model.fit(
    X_train, y_train,
    cat_features=cat,
    eval_set=(X_val, y_val),
    logging_level='Verbose',
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.112392
0:	learn: 0.9006992	test: 0.9039439	best: 0.9039439 (0)	total: 162ms	remaining: 2m 41s
1:	learn: 0.7585272	test: 0.7634507	best: 0.7634507 (1)	total: 219ms	remaining: 1m 49s
2:	learn: 0.6485665	test: 0.6550057	best: 0.6550057 (2)	total: 250ms	remaining: 1m 23s
3:	learn: 0.5618393	test: 0.5691495	best: 0.5691495 (3)	total: 286ms	remaining: 1m 11s
4:	learn: 0.4905922	test: 0.4992432	best: 0.4992432 (4)	total: 334ms	remaining: 1m 6s
5:	learn: 0.4329113	test: 0.4424857	best: 0.4424857 (5)	total: 385ms	remaining: 1m 3s
6:	learn: 0.3830413	test: 0.3943234	best: 0.3943234 (6)	total: 438ms	remaining: 1m 2s
7:	learn: 0.3415059	test: 0.3534914	best: 0.3534914 (7)	total: 481ms	remaining: 59.6s
8:	learn: 0.3060221	test: 0.3192112	best: 0.3192112 (8)	total: 526ms	remaining: 58s
9:	learn: 0.2752674	test: 0.2894858	best: 0.2894858 (9)	total: 565ms	remaining: 55.9s
10:	learn: 0.2483950	test: 0.2637211	best: 0.2637211 (10)	total: 597ms	remaining: 53.7s
11:	learn: 0.2251603

# Одноклассовая сегментация + классификация + катбуст

In [136]:
# вал
d = model.eval_metrics(
    Pool(X_val, y_val, cat_features=cat),
    metrics=[metrics.Accuracy(),metrics.AUC()]
)
for k in d:
    print(k, np.mean(d[k]), np.std(d[k]))

Accuracy 0.9747462028803022 0.0009344062253286436
AUC:type=Mu 0.9986557771491663 0.0004997685314638939


In [134]:
# свой тест
d = model.eval_metrics(
    Pool(X_test, y_test, cat_features=cat),
    metrics=[metrics.Accuracy(),metrics.AUC()]
)
for k in d:
    print(k, np.mean(d[k]), np.std(d[k]))

Accuracy 0.979381443298969 0.0034183559817920614
AUC:type=Mu 0.9990186091182356 0.0007899761034591391


In [135]:
# сабмит с хакатона
d = model.eval_metrics(
    Pool(X_submit, y_submit, cat_features=cat),
    metrics=[metrics.Accuracy(),metrics.AUC()]
)
for k in d:
    print(k, np.mean(d[k]), np.std(d[k]))

Accuracy 0.9003985099194317 0.0017047260070164854
AUC:type=Mu 0.9726438754104939 0.0013892412022269136


In [116]:
pd.Series(data=model.get_feature_importance(), index=columns).sort_values(ascending=False).head(200)

shipun p 1    35.073958
maliy p 1     20.538601
klikun p 1    20.465967
pred 1         8.517595
seg conf 1     6.061926
mul conf 1     1.952376
cls conf 1     1.757872
cls max        1.713921
maliy p 2      0.763907
klikun p 2     0.708051
pred 2         0.624692
pred 4         0.443025
pred 3         0.369732
shipun p 2     0.157238
maliy p 3      0.152981
mul conf 2     0.133117
seg max        0.116023
seg conf 2     0.090702
common         0.086607
cls conf 2     0.060995
shipun p 3     0.036830
seg conf 4     0.032684
shipun p 4     0.029068
maliy p 4      0.018137
klikun p 3     0.018091
cls conf 4     0.010044
cls conf 3     0.009810
maliy p 5      0.009319
klikun p 4     0.008064
seg conf 3     0.007841
pred 5         0.007458
mul conf 3     0.006009
mul conf 5     0.005461
cls conf 5     0.005042
klikun p 5     0.003949
shipun p 5     0.002907
seg conf 5     0.000000
mul conf 4     0.000000
dtype: float64